In [8]:
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib.pyplot import sca

from experiments.evaluation_flow.shared import GOOGLENET, RESNET_152, MOBILENET, RESNET_18, RESNET_50
from experiments.plots.util import *
import matplotlib
import matplotlib.lines as mlines

In [9]:
# define the directory containing the experiment logsfrom experiments.util.extract_data import ns2s
BASELINE = 'baseline'
PARAM_UPDATE = 'param_update'
PARAM_UPDATE_IMPROVED = 'param_update_improved'
PROVENANCE = 'provenance'

VERSION = 'version'
FINE_TUNED = 'fine-tuned'

ROOT_LOG_DIR = '/Users/nils/uni/mmlib-paper/master-thesis/experiments/plots/dist-exp/results/storage-consumption-select'

In [10]:
# the storage consumption is determined on the server -> filter files that are server logs
all_files = all_files_in_dir(ROOT_LOG_DIR)
server_files = [f for f in all_files if 'server' in f]

In [11]:
U_IDS = 'u_ids'
# for all files extract the metadata (e.g. what model and dataset is used)
server_files_and_meta = [(extract_file_meta(f), f) for f in server_files]
# add a mapping: use_case -> model id
server_files_and_meta = [({**f[0], **{U_IDS: use_case_ids(f[1])}}, f[1]) for f in server_files_and_meta]

In [12]:
# add the total storage consumption, and the detailed storage consumption to the metadata
# and save all metadata as a list in the variable storage_meta
storage_meta = []
for f in server_files_and_meta:
    use_case_id_dict = f[0][U_IDS]
    consumptions = {}
    total_consumptions = {}
    for use_case in use_case_id_dict.keys():
        s_info = storage_info(f[1], use_case_id_dict[use_case])
        consumptions[use_case] = s_info
        total_consumptions[use_case] = total_storage_consumption(s_info)

    f[0].update({CONSUMPTIONS: consumptions, TOTAL_CONSUMPTIONS: total_consumptions})
    storage_meta.append(f[0])

In [13]:
def print_cons_first_elem_mobilenet_outdoor(approach, snapshot_type=VERSION):
    metas_base_outdoor = filter_meta(storage_meta, model=MOBILENET, approach=approach, snapshot_type=snapshot_type, snapshot_dist='outdoor')
    tmp_elem = metas_base_outdoor[0]
    consumption = tmp_elem[TOTAL_CONSUMPTIONS]
    print(consumption)

In [14]:
# metas for baseline outdoor
print('baseline')
print_cons_first_elem_mobilenet_outdoor(BASELINE)


baseline
{'U_1': 14345728, 'N0_U_3_1_1': 14345771, 'N1_U_3_1_1': 14345771, 'N2_U_3_1_1': 14345771, 'N3_U_3_1_1': 14345771, 'N4_U_3_1_1': 14345771, 'N5_U_3_1_1': 14345771, 'N6_U_3_1_1': 14345771, 'N7_U_3_1_1': 14345771, 'N8_U_3_1_1': 14345771, 'N9_U_3_1_1': 14345771, 'N10_U_3_1_1': 14345771, 'N11_U_3_1_1': 14345771, 'N12_U_3_1_1': 14345771, 'N13_U_3_1_1': 14345771, 'N14_U_3_1_1': 14345771, 'N15_U_3_1_1': 14345771, 'N16_U_3_1_1': 14345771, 'N17_U_3_1_1': 14345771, 'N18_U_3_1_1': 14345771, 'N19_U_3_1_1': 14345771, 'N0_U_3_1_2': 14345771, 'N1_U_3_1_2': 14345771, 'N2_U_3_1_2': 14345771, 'N3_U_3_1_2': 14345771, 'N4_U_3_1_2': 14345771, 'N5_U_3_1_2': 14345771, 'N6_U_3_1_2': 14345771, 'N7_U_3_1_2': 14345771, 'N8_U_3_1_2': 14345771, 'N9_U_3_1_2': 14345771, 'N10_U_3_1_2': 14345771, 'N11_U_3_1_2': 14345771, 'N12_U_3_1_2': 14345771, 'N13_U_3_1_2': 14345771, 'N14_U_3_1_2': 14345771, 'N15_U_3_1_2': 14345771, 'N16_U_3_1_2': 14345771, 'N17_U_3_1_2': 14345771, 'N18_U_3_1_2': 14345771, 'N19_U_3_1_2': 143

In [15]:
# metas for update outdoor
print('update - version')
print_cons_first_elem_mobilenet_outdoor(PARAM_UPDATE_IMPROVED)

update - version
{'U_1': 14345728, 'N0_U_3_1_1': 14327222, 'N1_U_3_1_1': 14327222, 'N2_U_3_1_1': 14327222, 'N3_U_3_1_1': 14327222, 'N4_U_3_1_1': 14327222, 'N5_U_3_1_1': 14327222, 'N6_U_3_1_1': 14327222, 'N7_U_3_1_1': 14327222, 'N8_U_3_1_1': 14327222, 'N9_U_3_1_1': 14327222, 'N10_U_3_1_1': 14327222, 'N11_U_3_1_1': 14327222, 'N12_U_3_1_1': 14327222, 'N13_U_3_1_1': 14327222, 'N14_U_3_1_1': 14327222, 'N15_U_3_1_1': 14327222, 'N16_U_3_1_1': 14327222, 'N17_U_3_1_1': 14327222, 'N18_U_3_1_1': 14327222, 'N19_U_3_1_1': 14327222, 'N0_U_3_1_2': 14327222, 'N1_U_3_1_2': 14327222, 'N2_U_3_1_2': 14327222, 'N3_U_3_1_2': 14327222, 'N4_U_3_1_2': 14327222, 'N5_U_3_1_2': 14327222, 'N6_U_3_1_2': 14327222, 'N7_U_3_1_2': 14327222, 'N8_U_3_1_2': 14327222, 'N9_U_3_1_2': 14327222, 'N10_U_3_1_2': 14327222, 'N11_U_3_1_2': 14327222, 'N12_U_3_1_2': 14327222, 'N13_U_3_1_2': 14327222, 'N14_U_3_1_2': 14327222, 'N15_U_3_1_2': 14327222, 'N16_U_3_1_2': 14327222, 'N17_U_3_1_2': 14327222, 'N18_U_3_1_2': 14327222, 'N19_U_3_1

In [16]:
# metas for update outdoor
print('update - fine-tuned')
print_cons_first_elem_mobilenet_outdoor(PARAM_UPDATE_IMPROVED, FINE_TUNED)


update - fine-tuned
{'U_1': 14345728, 'N0_U_3_1_1': 5195070, 'N1_U_3_1_1': 5195070, 'N2_U_3_1_1': 5195070, 'N3_U_3_1_1': 5195070, 'N4_U_3_1_1': 5195070, 'N5_U_3_1_1': 5195070, 'N6_U_3_1_1': 5195070, 'N7_U_3_1_1': 5195070, 'N8_U_3_1_1': 5195070, 'N9_U_3_1_1': 5195070, 'N10_U_3_1_1': 5195070, 'N11_U_3_1_1': 5195070, 'N12_U_3_1_1': 5195070, 'N13_U_3_1_1': 5195070, 'N14_U_3_1_1': 5195070, 'N15_U_3_1_1': 5195070, 'N16_U_3_1_1': 5195070, 'N17_U_3_1_1': 5195070, 'N18_U_3_1_1': 5195070, 'N19_U_3_1_1': 5195070, 'N0_U_3_1_2': 5195070, 'N1_U_3_1_2': 5195070, 'N2_U_3_1_2': 5195070, 'N3_U_3_1_2': 5195070, 'N4_U_3_1_2': 5195070, 'N5_U_3_1_2': 5195070, 'N6_U_3_1_2': 5195070, 'N7_U_3_1_2': 5195070, 'N8_U_3_1_2': 5195070, 'N9_U_3_1_2': 5195070, 'N10_U_3_1_2': 5195070, 'N11_U_3_1_2': 5195070, 'N12_U_3_1_2': 5195070, 'N13_U_3_1_2': 5195070, 'N14_U_3_1_2': 5195070, 'N15_U_3_1_2': 5195070, 'N16_U_3_1_2': 5195070, 'N17_U_3_1_2': 5195070, 'N18_U_3_1_2': 5195070, 'N19_U_3_1_2': 5195070, 'N0_U_3_1_3': 5195070,

In [17]:
# metas for prov outdoor
print('provenance')
print_cons_first_elem_mobilenet_outdoor(PROVENANCE)


provenance
{'U_1': 14345728, 'N0_U_3_1_1': 72336625, 'N1_U_3_1_1': 72336625, 'N2_U_3_1_1': 72336625, 'N3_U_3_1_1': 72336625, 'N4_U_3_1_1': 72336625, 'N5_U_3_1_1': 72336625, 'N6_U_3_1_1': 72336625, 'N7_U_3_1_1': 72336625, 'N8_U_3_1_1': 72336625, 'N9_U_3_1_1': 72336625, 'N10_U_3_1_1': 72336625, 'N11_U_3_1_1': 72336625, 'N12_U_3_1_1': 72336625, 'N13_U_3_1_1': 72336625, 'N14_U_3_1_1': 72336625, 'N15_U_3_1_1': 72336625, 'N16_U_3_1_1': 72336625, 'N17_U_3_1_1': 72336625, 'N18_U_3_1_1': 72336625, 'N19_U_3_1_1': 72336625, 'N0_U_3_1_2': 72336625, 'N1_U_3_1_2': 72336625, 'N2_U_3_1_2': 72336625, 'N3_U_3_1_2': 72336625, 'N4_U_3_1_2': 72336625, 'N5_U_3_1_2': 72336625, 'N6_U_3_1_2': 72336625, 'N7_U_3_1_2': 72336625, 'N8_U_3_1_2': 72336625, 'N9_U_3_1_2': 72336625, 'N10_U_3_1_2': 72336625, 'N11_U_3_1_2': 72336625, 'N12_U_3_1_2': 72336625, 'N13_U_3_1_2': 72336625, 'N14_U_3_1_2': 72336625, 'N15_U_3_1_2': 72336625, 'N16_U_3_1_2': 72336625, 'N17_U_3_1_2': 72336625, 'N18_U_3_1_2': 72336625, 'N19_U_3_1_2': 7